In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, '../../')
from library.notebook_api.data_loader import  ModelDataLoader,CombinedDataLoader
from configuration import PROJECT_ABSOLUTE_PATH,DATA_SOURCE_FOLDER, DATA_SOURCE_PATH, MODEL_INPUT_DATA_PATH,FMA_METADATA_PATH,FMA_AUDIO_PATH,GTZAN_METADATA_PATH,GTZAN_AUDIO_PATH


### Combined data loader for all meta available as dataframes

In [2]:
data_loader = CombinedDataLoader()
df = data_loader.df

In [3]:
df.count()

track_id          107574
dataset           107574
audio_path        107574
label              50598
fma_genre_top      50598
fma_genres        107574
fma_genres_all    107574
file_available      9000
dtype: int64

In [4]:
df.head()

,track_id,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,file_available
0,002,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],NaN
1,003,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],NaN
2,005,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],NaN
3,010,fma,/project_data_source/free_music_archive/fma_sm...,pop,Pop,[10],[10],NaN
4,020,fma,/project_data_source/free_music_archive/fma_sm...,NaN,NaN,"[76, 103]","[17, 10, 76, 103]",NaN


### Model data loader for loading pre-processed features based on the fiels

In [6]:
model_data = ModelDataLoader('005')

In [7]:
model_data.df.count()

index                                 8989
track_id                              8989
dataset                               8989
audio_path                            8989
label                                 8989
fma_genre_top                         8989
fma_genres                            8989
fma_genres_all                        8989
file_available                        8989
sampling_rate                         8989
features                              8989
spectral_centroids_mean               8989
spectral_centroids_delta_mean         8989
spectral_centroids_accelerate_mean    8989
spectral_bandwidth_mean               8989
spectral_rolloff_mean                 8989
zero_crossing_rate_mean               8989
rms_mean                              8989
chroma_stft_mean                      8989
mfccs_mean                            8989
onset                                 8989
tempo                                 8989
contrast                              8989
tonnetz    

In [9]:
model_data.feature_names

['spectral_centroids_mean',
 'spectral_centroids_delta_mean',
 'spectral_centroids_accelerate_mean',
 'spectral_bandwidth_mean',
 'spectral_rolloff_mean',
 'zero_crossing_rate_mean',
 'rms_mean',
 'chroma_stft_mean',
 'mfccs_mean']

In [8]:
model_data.df.head()

,index,track_id,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,file_available,sampling_rate,...,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean,onset,tempo,contrast,tonnetz,mfccs_min,mfccs_max
0,0,000002,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],1.0,44100,...,0.098364,0.141225,0.424169,-1.742610,1.403928,166.708664,21.300203,0.022087,-505.499176,196.868652
1,2,000005,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],1.0,44100,...,0.059334,0.145243,0.475249,1.318397,1.384270,99.384018,21.096722,0.005648,-528.116272,209.443054
2,3,000010,fma,/project_data_source/free_music_archive/fma_sm...,pop,Pop,[10],[10],1.0,44100,...,0.080989,0.186656,0.430406,2.244156,1.123551,112.347145,20.852177,-0.033316,-537.197205,229.155151
3,15,000140,fma,/project_data_source/free_music_archive/fma_sm...,folk,Folk,[17],[17],1.0,44100,...,0.027256,0.068459,0.450882,-3.624865,1.135973,107.666016,21.727451,0.012266,-592.895813,199.950806
4,16,000141,fma,/project_data_source/free_music_archive/fma_sm...,folk,Folk,[17],[17],1.0,44100,...,0.034120,0.100852,0.341356,-4.471582,0.881413,120.185318,23.654802,-0.003315,-536.546936,222.208649


In [9]:
#lables
model_data.df.label.unique()

array(['hiphop', 'pop', 'folk', 'experimental', 'rock', 'international',
       'electronic', 'instrumental', 'blues', 'classical', 'country',
       'soulrnb', 'jazz'], dtype=object)

In [10]:
#feature names
model_data.feature_names

['spectral_centroids_mean',
 'spectral_centroids_delta_mean',
 'spectral_centroids_accelerate_mean',
 'spectral_bandwidth_mean',
 'spectral_rolloff_mean',
 'zero_crossing_rate_mean',
 'rms_mean',
 'chroma_stft_mean',
 'mfccs_mean',
 'onset',
 'tempo',
 'contrast',
 'tonnetz',
 'mfccs_min',
 'mfccs_max']

In [15]:
#features
model_data.df[model_data.feature_names].head()

,spectral_centroids_mean,spectral_centroids_delta_mean,spectral_centroids_accelerate_mean,spectral_bandwidth_mean,spectral_rolloff_mean,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean
0,3683.997559,1.232585,-0.456871,3495.670410,7419.114258,0.098364,0.141225,0.424169,-1.742610
1,2928.398682,0.680911,0.022614,3383.928955,6355.410156,0.059334,0.145243,0.475249,1.318397
2,2786.793457,-0.854561,-0.076388,2895.759033,5242.592773,0.080989,0.186656,0.430406,2.244156
3,1957.348267,1.383136,0.351598,2927.964355,4312.445312,0.027256,0.068459,0.450882,-3.624865
4,1768.606567,1.122301,-0.099131,2358.308838,3492.715576,0.034120,0.100852,0.341356,-4.471582


In [3]:
model_data_small = ModelDataLoader('000')

In [4]:
model_data_small.df

,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,librosa_load,sampling_rate,features,spectral_centroids_mean,spectral_centroids_delta_mean,spectral_centroids_accelerate_mean,spectral_bandwidth_mean,spectral_rolloff_mean,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean
track_id,,,,,,,,,,,,,,,,,,
blues.00000.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0073242188, 0.016601562, 0.0076293945, -0.0...",22050,"[1784.1227, -0.49052292, -0.061881434, 2002.41...",1784.122681,-0.490523,-0.061881,2002.412354,3805.723145,0.083045,0.130184,0.350129,3.121230
blues.00001.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0034179688, 0.0043029785, 0.001373291, 0.00...",22050,"[1530.2617, 0.6459638, -0.0855879, 2038.9875, ...",1530.261719,0.645964,-0.085588,2038.987549,3550.713623,0.056040,0.095908,0.340849,-0.007777
blues.00002.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.019012451, 0.047698975, 0.029418945, -0.018...",22050,"[1552.8325, 0.3400274, 0.025039379, 1747.754, ...",1552.832520,0.340027,0.025039,1747.754028,3042.410156,0.076291,0.175473,0.363538,3.364070
blues.00003.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.013000488, -0.03060913, -0.036071777, -0.0...",22050,"[1070.1534, 0.25125474, 0.07936938, 1596.4226,...",1070.153442,0.251255,0.079369,1596.422607,2184.879150,0.033309,0.141040,0.404854,0.866870
blues.00004.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.0063171387, -0.009277344, -0.008331299, -0...",22050,"[1835.1285, -0.17207043, -0.24526799, 1748.410...",1835.128540,-0.172070,-0.245268,1748.410767,3579.957520,0.101461,0.091501,0.308526,-9.653034
